In [1]:
from music21 import *
import tensorflow
import sys
import os
import IPython
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
import keras
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import random

In [2]:
environment.set('musescoreDirectPNGPath', 'MuseScore-4.0.1.230121751-x86_64.AppImage')

In [3]:
filepath = "./"

In [6]:
all_midis= []
names = []
j = 0
for i in os.listdir(filepath):
    if j > 100:
      break
    if i.endswith(".mid"):
        tr = filepath + i
        print(tr)
        names.append(tr)
        midi = converter.parse(tr)
        all_midis.append(midi)
    j += 1
        

./Spice_Girls_-_Viva_Forever.mid
./Bee_Gees_-_To_Love_Somebody.mid
./x (9).mid
./Christina_Aguilera_-_Come_On_Over.mid
./Blur_-_Charmless_Man.mid
./Enya_-_Bard_Dance.mid
./The_Temptations_-_Get_Ready.mid
./x (29).mid
./x (14).mid
./Harry_Chapin_-_Cats_In_The_Cradle.mid
./Oasis_-_Live_Forever.mid
./Michael_Jackson_-_Beat_It.mid
./Phil_Collins_-_In_the_Air_Tonight.mid
./Marvin_Gaye_-_Mercy_Mercy_Me.mid
./Mason_Williams_-_Classical_Gas.mid
./x (32).mid
./Mary_Wells_-_My_Guy.mid
./Oasis_-_Talk_Tonight.mid
./Celine_Dion_-_Falling_Into_You.mid
./Chicago_-_Does_Anybody_Really_Know.mid
./x (15).mid
./Oasis_-_Whatever.mid
./x (37).mid
./Bee_Gees_-_Stayin_Alive.mid
./The_Monkees_-_I'm_A_Believer.mid
./Mariah_Carey_-_Thank_God_I_Found_You.mid
./Bee_Gees_-_More_Than_A_Woman.mid
./Paul_McCartney_-_Band_On_The_Run.mid
./Marc_Anthony_-_I_Need_to_Know.mid
./Brian_McKnight_-_Back_At_One.mid
./Tina_Turner_-_Nutbush_City_Limits.mid
./Destinys_Child_-_Jumpin_Jumpin.mid
./x (38).mid
./Lionel_Richie_-_Endle

In [7]:
def extract_notes(files):
    notes = []
    pick = None
    for f in files:
        #songs = instrument.partitionByInstrument(f)
        for part in f.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes
#Getting the list of notes as Corpus
Corpus= np.array(extract_notes(all_midis))
#print("Total notes in all the Mozzart midis in the dataset:", len(Corpus))

In [8]:
Corpus = np.reshape(Corpus, (len(Corpus), 1))

In [9]:
Corpus.shape

(165267, 1)

In [10]:
Corpus

array([['3.6.10'],
       ['3.6.10'],
       ['10.1.5'],
       ...,
       ['9.1.4'],
       ['9.1.4'],
       ['9.1.4']], dtype='<U25')

In [11]:
len(list(filter(lambda x : x == 'C3', Corpus)))

1348

In [12]:
# Initialize OneHotEncoder object
encoder = OneHotEncoder()

# Fit and transform the data
one_hot_data = encoder.fit_transform(Corpus)

# Convert sparse matrix to array
one_hot_data = one_hot_data.toarray()

# Print the one-hot encoded data
print(one_hot_data)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [13]:
one_hot_data.shape

(165267, 555)

In [14]:
input_len = 5

In [15]:
def getXy(data, step):
    y_ind = np.arange(step, len(data), step)
    y = data[y_ind]
    rows_x = len(y)
    X = data[range(rows_x * step)]
    X = np.reshape(X, (rows_x, step, one_hot_data.shape[1]))
    return X, y

In [16]:
print(len(one_hot_data))
indeksi = np.arange(input_len, len(one_hot_data),input_len)
one_hot_data[indeksi].shape

165267


(33053, 555)

In [17]:
X, y = getXy(one_hot_data, input_len)

In [18]:
X.shape

(33053, 5, 555)

In [19]:
y.shape

(33053, 555)

In [20]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(LSTM(hidden_units, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(int(hidden_units/2)))
    model.add(Dense(units=dense_units))
    model.add(Dropout(0.1))
    model.add(Dense(units=one_hot_data.shape[1], activation='softmax'))
    model.compile(loss=tensorflow.losses.CategoricalCrossentropy(), optimizer='adam')
    return model

In [21]:
model = create_RNN(516, 164, input_shape=(input_len, one_hot_data.shape[1]), activation=['softmax', 'softmax'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 516)            2212608   
                                                                 
 dropout (Dropout)           (None, 5, 516)            0         
                                                                 
 lstm_1 (LSTM)               (None, 258)               799800    
                                                                 
 dense (Dense)               (None, 164)               42476     
                                                                 
 dropout_1 (Dropout)         (None, 164)               0         
                                                                 
 dense_1 (Dense)             (None, 555)               91575     
                                                                 
Total params: 3,146,459
Trainable params: 3,146,459
Non-

In [23]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
len(np.unique(X))

2

In [25]:
model.fit(X, y, epochs=10)

Epoch 1/10
1033/1033 [==============================] - 146s 137ms/step - loss: 3.3655
Epoch 2/10
1033/1033 [==============================] - 142s 138ms/step - loss: 2.6635
Epoch 3/10
1033/1033 [==============================] - 150s 146ms/step - loss: 2.4806
Epoch 4/10
1033/1033 [==============================] - 144s 139ms/step - loss: 2.3417
Epoch 5/10
1033/1033 [==============================] - 143s 139ms/step - loss: 2.2079
Epoch 6/10
1033/1033 [==============================] - 145s 140ms/step - loss: 2.0566
Epoch 7/10
1033/1033 [==============================] - 148s 143ms/step - loss: 1.9040
Epoch 8/10
1033/1033 [==============================] - 148s 143ms/step - loss: 1.7403
Epoch 9/10
1033/1033 [==============================] - 148s 143ms/step - loss: 1.5763
Epoch 10/10
1033/1033 [==============================] - 147s 142ms/step - loss: 1.4022


In [70]:
scales = {
  "c-major": ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', '0.4.7','4.7.11', '2.5.9', '5.9.0', '7.11.2', '9.0.4'],
  "c#-major" : ['A#3', 'C3', 'C#3', 'D#3', 'F3', 'F#3', 'G#3', '1.5.8','5.8.12', '3.6.10', '6.10.0', '8.0.3', '10.1.5'],
  "d-major": ['B3', 'C#3', 'D3', 'E3', 'F#3', 'G3', 'A3', '2.6.9','6.9.1', '4.7.11', '7.11.2', '9.1.4', '11.2.6'],
  "d#-major": ['C3', 'D3', 'D#3', 'F3', 'G3', 'G#3', 'A#3', '3.7.10','7.10.2', '5.8.0', '8.0.3', '10.2.5', '10.1.5'],
  "e-major": ['C#3', 'D#3', 'E3', 'F#3', 'G#3', 'A3', 'B3', '4.8.11','8.11.3', '6.9.1', '9.1.4', '11.3.6', '11.2.6'],
  "f-major": ['D3', 'E3', 'F3', 'G3', 'A3', 'A#3', 'C3', '5.9.0','9.0.4', '7.10.2', '10.2.5', '0.4.7', '0.3.7'],
  "f#-major": ['D#3', 'F3', 'F#3', 'G#3', 'A#3', 'B3', 'C#3', '6.10.1','10.1.5', '8.11.3', '11.3.6', '1.5.8', '1.4.8'],
  "g-major": ['E3', 'F#3', 'G3', 'A3', 'B3', 'C3', 'D3', '7.11.2','11.2.6', '9.0.4', '0.4.7', '2.6.9', '2.5.9'],
  "g#-major": ['F3', 'G3', 'G#3', 'A#3', 'C3', 'C#3', 'D#3', '8.0.3','0.3.7', '10.1.5', '1.5.8', '3.7.10', '3.6.10'],
  "a-major": ['F#3', 'G#3', 'A3', 'B3', 'C#3', 'D3', 'E3', '9.1.4','1.4.8', '11.2.6', '2.6.9', '4.8.11', '4.7.11'],
  "a#-major": ['G3', 'A3', 'A#3', 'C3', 'D3', 'D#3', 'F3', '10.2.5','2.5.9', '0.3.7', '3.7.10', '5.9.0', '5.8.0'],
  "b-major": ['G#3', 'A#3', 'B3', 'C#3', 'D#3', 'E3', 'F#3', '11.3.6','3.6.10', '1.4.8', '4.8.11', '6.10.1', '6.9.1'],
  
  "a-minor": ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', '0.4.7','4.7.11', '2.5.9', '5.9.0', '7.11.2', '9.0.4'],
  "a#-minor" : ['A#3', 'C3', 'C#3', 'D#3', 'F3', 'F#3', 'G#3', '1.5.8','5.8.12', '3.6.10', '6.10.0', '8.0.3', '10.1.5'],
  "b-minor": ['B3', 'C#3', 'D3', 'E3', 'F#3', 'G3', 'A3', '2.6.9','6.9.1', '4.7.11', '7.11.2', '9.1.4', '11.2.6'],
  "c-minor": ['C3', 'D3', 'D#3', 'F3', 'G3', 'G#3', 'A#3', '3.7.10','7.10.2', '5.8.0', '8.0.3', '10.2.5', '10.1.5'],
  "c#-minor": ['C#3', 'D#3', 'E3', 'F#3', 'G#3', 'A3', 'B3', '4.8.11','8.11.3', '6.9.1', '9.1.4', '11.3.6', '11.2.6'],
  "d-minor": ['D3', 'E3', 'F3', 'G3', 'A3', 'A#3', 'C3', '5.9.0','9.0.4', '7.10.2', '10.2.5', '0.4.7', '0.3.7'],
  "d#-minor": ['D#3', 'F3', 'F#3', 'G#3', 'A#3', 'B3', 'C#3', '6.10.1','10.1.5', '8.11.3', '11.3.6', '1.5.8', '1.4.8'],
  "e-minor": ['E3', 'F#3', 'G3', 'A3', 'B3', 'C3', 'D3', '7.11.2','11.2.6', '9.0.4', '0.4.7', '2.6.9', '2.5.9'],
  "f-minor": ['F3', 'G3', 'G#3', 'A#3', 'C3', 'C#3', 'D#3', '8.0.3','0.3.7', '10.1.5', '1.5.8', '3.7.10', '3.6.10'],
  "f#-minor": ['F#3', 'G#3', 'A3', 'B3', 'C#3', 'D3', 'E3', '9.1.4','1.4.8', '11.2.6', '2.6.9', '4.8.11', '4.7.11'],
  "g-minor": ['G3', 'A3', 'A#3', 'C3', 'D3', 'D#3', 'F3', '10.2.5','2.5.9', '0.3.7', '3.7.10', '5.9.0', '5.8.0'],
  "g#-minor": ['G#3', 'A#3', 'B3', 'C#3', 'D#3', 'E3', 'F#3', '11.3.6','3.6.10', '1.4.8', '4.8.11', '6.10.1', '6.9.1'],
}


In [166]:
input_scale = input('choose scale')

choose scalec-major


In [189]:
new_corpus = np.array([random.sample(scales[input_scale], 5)])

In [190]:
new_corpus.shape

(1, 5)

In [191]:
new_corpus = np.reshape(new_corpus, (len(new_corpus[0]), 1))

In [192]:
new_corpus.shape

(5, 1)

In [193]:
one_hot_new_data = encoder.transform(new_corpus)
one_hot_new_data = one_hot_new_data.toarray()

In [194]:
one_hot_new_data.shape

(5, 555)

In [195]:
def funkcija(broj):
    if broj > 0.2:
        return 1
    else:
        return 0

In [196]:
s = set({})

In [197]:
1 == 1.0

True

In [198]:
def three_in_a_row(new_note, one_hot_new_data, i):
    entry = encoder.inverse_transform(new_note)
    last = encoder.inverse_transform(one_hot_new_data[i-1:i])
    before_last = encoder.inverse_transform(one_hot_new_data[i-2:i-1])
    if entry != last or entry != before_last:
        return False
    return True

In [199]:
def choose_another_note(forbidden):
    our_scale = set(scales[input_scale])
    if np.any(forbidden):
        forbidden = np.reshape(forbidden, (1, -1))
        forbidden = encoder.inverse_transform(forbidden)
        our_scale_valid = our_scale.difference({forbidden[0][0]})
    else:
        our_scale_valid = our_scale
    new_note = np.array([random.choice(list(our_scale_valid))])
    new_note = np.reshape(new_note, (len(new_note), 1))

    new_note = encoder.transform(new_note)
    new_note = new_note.toarray()
    
    return new_note
    
    

In [200]:
def in_scale(new_note):
    our_scale = set(scales[input_scale])
    if encoder.inverse_transform(new_note)[0][0] not in our_scale:
        return False
    return True

In [201]:
one_hot_new_data.shape

(5, 555)

In [202]:
i = input_len

In [203]:
while i < 112:
    new_note = model.predict(np.reshape(one_hot_new_data[i-input_len:i], (1, input_len, one_hot_data.shape[1])))
    new_note = np.array(list(map(funkcija, new_note[0])))
    new_note = np.reshape(new_note, (1, one_hot_data.shape[1]))
    
    
    while not np.any(new_note[0]) or not in_scale(new_note) or three_in_a_row(new_note, one_hot_new_data, i):
        new_note = choose_another_note(new_note[0])
        new_note = np.array(list(map(funkcija, new_note[0])))
        new_note = np.reshape(new_note, (1, one_hot_data.shape[1]))
    one_hot_new_data = np.append(one_hot_new_data, new_note, axis=0)
    i += 1



1/1 [==============================] - 0s 27ms/step


In [204]:
one_hot_new_data = encoder.inverse_transform(one_hot_new_data)

In [205]:
one_hot_new_data

array([['0.4.7'],
       ['G3'],
       ['2.5.9'],
       ['C3'],
       ['B3'],
       ['D3'],
       ['D3'],
       ['E3'],
       ['E3'],
       ['9.0.4'],
       ['9.0.4'],
       ['G3'],
       ['A3'],
       ['A3'],
       ['F3'],
       ['A3'],
       ['A3'],
       ['2.5.9'],
       ['A3'],
       ['A3'],
       ['G3'],
       ['A3'],
       ['B3'],
       ['A3'],
       ['A3'],
       ['7.11.2'],
       ['D3'],
       ['A3'],
       ['A3'],
       ['0.4.7'],
       ['A3'],
       ['A3'],
       ['7.11.2'],
       ['7.11.2'],
       ['F3'],
       ['0.4.7'],
       ['0.4.7'],
       ['9.0.4'],
       ['0.4.7'],
       ['0.4.7'],
       ['A3'],
       ['0.4.7'],
       ['0.4.7'],
       ['9.0.4'],
       ['0.4.7'],
       ['0.4.7'],
       ['2.5.9'],
       ['0.4.7'],
       ['0.4.7'],
       ['4.7.11'],
       ['0.4.7'],
       ['0.4.7'],
       ['F3'],
       ['5.9.0'],
       ['0.4.7'],
       ['0.4.7'],
       ['5.9.0'],
       ['5.9.0'],
       ['0.4.7'],
       ['0.4.7'],


In [206]:
one_hot_new_data.shape

(112, 1)

In [207]:
def chords_n_notes(Snippet):
    Melody = []
    offset = 0 #Incremental
    for i in Snippet:
        #If it is chord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #Seperating the notes in chord
            notes = [] 
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)            
                notes.append(note_snip)
                chord_snip = chord.Chord(notes)
                chord_snip.offset = offset
                Melody.append(chord_snip)
        # pattern is a note
        else: 
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        # increase offset each iteration so that notes do not stack
        offset += 1
    Melody_midi = stream.Stream(Melody)   
    return Melody_midi

In [208]:
Melody = chords_n_notes(one_hot_new_data.reshape(112, ))

In [133]:
Melody_midy = stream.Stream(Melody)

In [209]:
Melody.write('midi','Melody_Generated_Better1.mid')

'Melody_Generated_Better1.mid'

In [ ]:
print("Sample Audio From Data")
IPython.display.Audio("Melody_Generated.mid") 


Sample Audio From Data


ValueError: rate must be specified when data is a numpy array or list of audio samples.